# Damage detection and recognition training pipeline

### Define run parameters
The dataset should follow the VGGFace2/ImageNet-style directory layout. Modify data_dir to the location of the dataset on wish to finetune on.

In [1]:
from facenet_pytorch import InceptionResnetV1, fixed_image_standardization, training
import torch
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import os
# from PIL import Image
# import copy
# import random
from functions import SiameseInceptionResnetV1, siamese_dataset, pass_epoch

In [2]:
data_dir = "C:/Users/U339700/Documents/Palas/dataset/prueba_siamesas/crop_2class_train"
val_dir = "C:/Users/U339700/Documents/Palas/dataset/prueba_siamesas/crop_2class_val"
model_dir = 'C:/Users/U339700/Documents/Palas/models'

batch_size = 32
epochs = 8
save_after = 1
workers = 0 if os.name == 'nt' else 8

### Determine if an nvidia GPU is available

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cpu


### Define Inception Resnet V1 module
See help(InceptionResnetV1) for more details.

Define dataset and dataloader

In [4]:
trans = transforms.Compose([
    transforms.Resize((256, 256)),
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

data_augm_horz = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(p=1),
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

data_augm_vert = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomVerticalFlip(p=1),
    transforms.ToTensor(),
    fixed_image_standardization
])

data_augm_persp = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomPerspective(distortion_scale=0.6, p=1.0),
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

data_augm_eq = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomEqualize(p=1),
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

data_augm_blur = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

data_augm_horz_vert = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(p=1),
    transforms.RandomVerticalFlip(p=1),
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

data_augm_horz_blur = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(p=1),
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

data_augm_vert_blur = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomVerticalFlip(p=1),
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

data_augm_horz_vert_blur = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(p=1),
    transforms.RandomVerticalFlip(p=1),
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])

In [4]:
train_dataset = siamese_dataset(data_dir, train=True, epoch_size=3000)
val_dataset = siamese_dataset(val_dir, train=False)

In [6]:
# train_dataset = siamese_dataset(data_dir, shuffle_pairs=True, augment=False)
# val_dataset = siamese_dataset(val_dir)

In [5]:
train_loader = DataLoader(train_dataset, batch_size=batch_size)
val_loader = DataLoader(val_dataset, batch_size=batch_size)

In [6]:
siamese = SiameseInceptionResnetV1(
    classify=False,
    pretrained='vggface2',
).to(device)

In [7]:
print(siamese)

SiameseInceptionResnetV1(
  (conv2d_1a): BasicConv2d(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2a): BasicConv2d(
    (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2d_2b): BasicConv2d(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (maxpool_3a): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2d_3b): BasicConv2d(
    (conv): Conv2d(64, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(80, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
  )
  (conv2

Define optimizer and scheduler

In [8]:
optimizer = optim.Adam(siamese.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])

### Define loss and evaluation functions

In [9]:
# loss_fn = torch.nn.CrossEntropyLoss()
loss_fn = torch.nn.BCELoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

In [10]:
# for i_batch, ((x1, x2), y, (class1, class2)) in enumerate(train_loader):
#     print(class2)

### Train model

In [10]:
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10
best_val = 10000000

print('\n\nInitial')
print('-' * 10)
siamese.eval()
pass_epoch(
    siamese, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    # Generate new random dataset
    train_dataset = siamese_dataset(data_dir, train=True, epoch_size=3000)
    train_loader = DataLoader(train_dataset, batch_size=batch_size)
    
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    # Train
    siamese.train()
    pass_epoch(
        siamese, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    # Validation
    siamese.eval()
    answ = pass_epoch(
        siamese, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )
    loss = answ[0]
    # Save model
    if loss < best_val:
        best_val = loss
        print('Saving best weights')
        torch.save(
            {
                "epoch": epoch + 1,
                "model_state_dict": siamese.state_dict(),
                # "backbone": args.backbone,
                "optimizer_state_dict": optimizer.state_dict()
            },
            os.path.join(model_dir, "best.pth")
        )            

    # Save model based on the frequency defined by "args.save_after"
    if (epoch + 1) % 2 == 0:
        torch.save(
            {
                "epoch": epoch + 1,
                "model_state_dict": siamese.state_dict(),
                # "backbone": args.backbone,
                "optimizer_state_dict": optimizer.state_dict()
            },
            os.path.join(model_dir, "epoch_{}.pth".format(epoch + 1))
        ) 

writer.close()



Initial
----------
Valid |     7/7    | loss:    0.7047 | fps:    0.6269 | acc:    0.4527   

Epoch 1/8
----------
Train |    94/94   | loss:    3.6439 | fps:    0.6760 | acc:    0.4989   
Valid |     7/7    | loss:   15.4857 | fps:    0.8517 | acc:    0.4527   
Saving best weights

Epoch 2/8
----------
Train |    94/94   | loss:    3.6213 | fps:    0.7265 | acc:    0.5014   
Valid |     7/7    | loss:    0.7118 | fps:    0.7487 | acc:    0.4527   
Saving best weights

Epoch 3/8
----------
Train |    94/94   | loss:    2.9474 | fps:    0.7198 | acc:    0.5055   
Valid |     7/7    | loss:   10.1382 | fps:    0.8372 | acc:    0.4527   

Epoch 4/8
----------
Train |    94/94   | loss:    2.6796 | fps:    0.7339 | acc:    0.4925   
Valid |     7/7    | loss:    8.8319 | fps:    0.7674 | acc:    0.4527   

Epoch 5/8
----------
Train |    94/94   | loss:    2.1853 | fps:    0.7325 | acc:    0.4961   
Valid |     7/7    | loss:    5.2990 | fps:    1.0258 | acc:    0.4527   

Epoch 6/8
----

In [81]:
torch.save(siamese, 'C:/Users/U339700/Documents/Palas/models/nuevo.pth')